<a href="https://colab.research.google.com/github/Byeonjinha/Colab/blob/main/%EC%9E%AC%EB%B0%8C%EB%8A%94%20%EA%B1%B0%20/T%EC%95%84%EC%B9%B4%EB%8D%B0%EB%AF%B8_%EC%9D%8C%EC%84%B1%ED%95%A9%EC%84%B1_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install torchaudio

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.5 MB/s eta 0:00:33tcmalloc: large alloc 1147494400 bytes == 0x5570f6d7e000 @  0x7f8936d7b615 0x5570bde6e4cc 0x5570bdf4e47a 0x5570bde712ed 0x5570bdf62e1d 0x5570bdee4e99 0x5570bdedf9ee 0x5570bde72bda 0x5570bdee4d00 0x5570bdedf9ee 0x5570bde72bda 0x5570bdee1737 0x5570bdf63c66 0x5570bdee0daf 0x5570bdf63c66 0x5570bdee0daf 0x5570bdf63c66 0x5570bdee0daf 0x5570bde73039 0x5570bdeb6409 0x5570bde71c52 0x5570bdee4c25 0x5570bdedf9ee 0x5570bde72bda 0x5570bdee1737 0x5570bdedf9ee 0x5570bde72bda 0x5570bdee0915 0x5570bde72afa 0x5570bdee0c0d 0x5570bdedf9ee
     |████████████████████████████████| 881.9 MB 20 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [2]:
import torch
import torch.nn as nn
import torchaudio
from torch import optim
from torch.utils.data import DataLoader
from torch.distributions.normal import Normal
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import librosa
import os
import argparse
import time, datetime
import json
import gc
from math import pi, log


print('Import 완료')



Import 완료


In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if not use_cuda:
    print('Edit - Notebook settings 에서 GPU로 변경해주세요.')

else:
    print('GPU 사용 준비가 완료되었습니다.')


GPU 사용 준비가 완료되었습니다.


In [4]:
LJSpeech_url = 'https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2'
train_dataset = torchaudio.datasets.LJSPEECH("", url=LJSpeech_url, download=True)

print('Download 완료')

  0%|          | 0.00/2.56G [00:00<?, ?B/s]

Download 완료


In [5]:
hop_length = 256
max_steps = 15872

class Collate():
    def __init__(self):
        self.wav_to_mel = torchaudio.transforms.MelSpectrogram(sample_rate=22050, n_mels=80, win_length=1024, hop_length=hop_length, f_min=0.0, f_max=8000.0, n_fft=1024)

    def __call__(self, batch):
        # batch: N_batch * [wav, sample_rate, text, text_normalized]

        x_batch = []
        c_batch = []
        for idx in range(len(batch)):
            x = batch[idx][0].squeeze()
            c = self.wav_to_mel(x)
            c = self.dynamic_range_compression(c)
            
            pad = (x.shape[0] // hop_length + 1) * hop_length - x.shape[0]
            pad_l = pad // 2
            pad_r = pad // 2 + pad % 2

            # zero pad for quantized signal
            x = np.pad(x, (pad_l, pad_r), mode="constant", constant_values=0.0)
            x = torch.tensor(x)

            max_time_frames = max_steps // hop_length
            s = np.random.randint(0, c.shape[1] - max_time_frames)
            ts = s * hop_length
            x = x[ts:ts + hop_length * max_time_frames]
            c = c[:,s:s + max_time_frames]
            
            x_batch.append(x)
            c_batch.append(c)

        x_batch = torch.stack(x_batch)
        c_batch = torch.stack(c_batch)

        return x_batch, c_batch

    def dynamic_range_compression(self, x, C=1, clip_val=1e-5):
        return torch.log(torch.clamp(x, min=clip_val) * C)

collate_fn = Collate()
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=False, collate_fn=collate_fn)
x_batch, c_batch = next(iter(train_loader))

print('audio batch shape:', x_batch.shape)
print('mel batch shape:', c_batch.shape)
print('Data 전처리 완료')

audio batch shape: torch.Size([8, 15872])
mel batch shape: torch.Size([8, 80, 62])
Data 전처리 완료


In [6]:
class UpsampleConv(nn.Module):
    def __init__(self, n_mel_channels=80):
        super().__init__()
        self.upsample = torch.nn.ConvTranspose1d(n_mel_channels, n_mel_channels, kernel_size=1024, stride=256)

    def forward(self, spect):
        spect = self.upsample(spect)
        return spect

In [7]:
class Invertible1x1Conv(torch.nn.Module):
    def __init__(self, c):
        super().__init__()
        self.conv = torch.nn.Conv1d(c, c, kernel_size=1, stride=1, padding=0, bias=False)
        W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
        if torch.det(W) < 0:
            W[:,0] = -1*W[:,0]
        W = W.view(c, c, 1)
        self.conv.weight.data = W

    def forward(self, x, log_det_sum):
        # shape
        z = self.conv(x)
        
        B, _, T = x.size()
        W = self.conv.weight.squeeze()
        log_det_W = B * T * torch.logdet(W)
        log_det_sum = log_det_sum + log_det_W

        return z, log_det_sum

    def reverse(self, z):
        W = self.conv.weight.squeeze()
        W_inverse = W.float().inverse()
        W_inverse = Variable(W_inverse[..., None])
        self.W_inverse = W_inverse
        x = F.conv1d(z, self.W_inverse, bias=None, stride=1, padding=0)
        
        return x

In [8]:
class WN(nn.Module):
    """
    This is the WaveNet like layer for the affine coupling.  The primary difference
    from WaveNet is the convolutions need not be causal.  There is also no dilation
    size reset.  The dilation only doubles on each layer
    """
    def __init__(self, in_channels, cin_channels, n_layers=8, n_channels=128,
                 kernel_size=3):
        super(WN, self).__init__()
        assert(kernel_size % 2 == 1)
        assert(n_channels % 2 == 0)
        self.n_layers = n_layers
        self.n_channels = n_channels
        self.in_layers = torch.nn.ModuleList()
        self.res_skip_layers = torch.nn.ModuleList()

        start = torch.nn.Conv1d(in_channels, n_channels, 1)
        start = torch.nn.utils.weight_norm(start, name='weight')
        self.start = start

        # Initializing last layer to 0 makes the affine coupling layers
        # do nothing at first.  This helps with training stability
        end = torch.nn.Conv1d(n_channels, 2*in_channels, 1)
        end.weight.data.zero_()
        end.bias.data.zero_()
        self.end = end

        cond_layer = torch.nn.Conv1d(cin_channels, 2*n_channels*n_layers, 1)
        self.cond_layer = torch.nn.utils.weight_norm(cond_layer, name='weight')

        for i in range(n_layers):
            dilation = 2 ** i
            padding = int((kernel_size*dilation - dilation)/2)
            in_layer = torch.nn.Conv1d(n_channels, 2*n_channels, kernel_size,
                                       dilation=dilation, padding=padding)
            in_layer = torch.nn.utils.weight_norm(in_layer, name='weight')
            self.in_layers.append(in_layer)

            # last one is not necessary
            if i < n_layers - 1:
                res_skip_channels = 2*n_channels
            else:
                res_skip_channels = n_channels
            res_skip_layer = torch.nn.Conv1d(n_channels, res_skip_channels, 1)
            res_skip_layer = torch.nn.utils.weight_norm(res_skip_layer, name='weight')
            self.res_skip_layers.append(res_skip_layer)

    def forward(self, audio, spect):
        audio = self.start(audio)
        output = torch.zeros_like(audio)
        n_channels_tensor = torch.IntTensor([self.n_channels])

        spect = self.cond_layer(spect)

        for i in range(self.n_layers):
            spect_offset = i*2*self.n_channels
            acts = self.fused_add_tanh_sigmoid_multiply(
                self.in_layers[i](audio),
                spect[:,spect_offset:spect_offset+2*self.n_channels,:],
                n_channels_tensor)

            res_skip_acts = self.res_skip_layers[i](acts)
            if i < self.n_layers - 1:
                audio = audio + res_skip_acts[:,:self.n_channels,:]
                output = output + res_skip_acts[:,self.n_channels:,:]
            else:
                output = output + res_skip_acts

        return self.end(output).chunk(2,1)


    def fused_add_tanh_sigmoid_multiply(self, input_a, input_b, n_channels):
        n_channels_int = n_channels[0]
        in_act = input_a+input_b
        t_act = torch.tanh(in_act[:, :n_channels_int, :])
        s_act = torch.sigmoid(in_act[:, n_channels_int:, :])
        acts = t_act * s_act
        return acts

In [9]:
class AffineCouplingLayer(nn.Module):
    def __init__(self, in_channels, cin_channels):
        super().__init__()
        self.WN = WN(in_channels, cin_channels)

    def forward(self, x, c, log_det_sum):

        x_a, x_b = x.chunk(2,1)
        log_s, t = self.WN(x_a, c)
        x_b = torch.exp(log_s) * x_b + t
        log_det_sum = log_det_sum + log_s.sum()
        z = torch.cat((x_a, x_b), dim=1)

        return z, c, log_det_sum

    def reverse(self, z, c):
        z_a, z_b = z.chunk(2,1)
        log_s, t = self.WN(z_a, c)
        z_b = torch.exp(-log_s) * (z_b - t)

        x = torch.cat((z_a, z_b), dim=1)

        return x, c


In [10]:
class WaveGlow(torch.nn.Module):
    def __init__(self, n_mel_channels=80, n_flows=12, n_group=8, n_early_every=4,
                 n_early_size=2):
        super(WaveGlow, self).__init__()

        self.upsample = UpsampleConv(n_mel_channels)
        assert(n_group % 2 == 0)
        self.n_flows = n_flows
        self.n_group = n_group
        self.n_early_every = n_early_every
        self.n_early_size = n_early_size
        self.coupling_layer = torch.nn.ModuleList()
        self.convinv = torch.nn.ModuleList()
        self.sigma = 1.0

        n_half = int(n_group/2)

        # Set up layers with the right sizes based on how many dimensions
        # have been output already
        n_remaining_channels = n_group
        for k in range(n_flows):
            if k % self.n_early_every == 0 and k > 0:
                n_half = n_half - int(self.n_early_size/2)
                n_remaining_channels = n_remaining_channels - self.n_early_size
            self.convinv.append(Invertible1x1Conv(n_remaining_channels))
            self.coupling_layer.append(AffineCouplingLayer(n_half, n_mel_channels*n_group))
        self.n_remaining_channels = n_remaining_channels  # Useful during inference

    def forward(self, audio, spect):
        audio = audio.squeeze(1)

        #  Upsample spectrogram to size of audio
        spect = self.upsample(spect)
        assert(spect.size(2) >= audio.size(1))
        if spect.size(2) > audio.size(1):
            spect = spect[:, :, :audio.size(1)]

        spect = spect.unfold(2, self.n_group, self.n_group).permute(0, 2, 1, 3)
        spect = spect.contiguous().view(spect.size(0), spect.size(1), -1).permute(0, 2, 1)

        audio = audio.unfold(1, self.n_group, self.n_group).permute(0, 2, 1)

        output_audio = []
        log_det_sum = 0.0

        for k in range(self.n_flows):
            if k % self.n_early_every == 0 and k > 0:
                output_audio.append(audio[:,:self.n_early_size,:])
                audio = audio[:,self.n_early_size:,:]
            audio, log_det_sum = self.convinv[k](audio, log_det_sum)
            audio, spect, log_det_sum = self.coupling_layer[k](audio, spect, log_det_sum)

        output_audio.append(audio)

        z = torch.cat(output_audio,1)

        log_p = 0.5 * (- log(2.0 * pi) - z.pow(2)).sum() / (z.size(0)*z.size(1)*z.size(2))
        log_det = log_det_sum /  (z.size(0)*z.size(1)*z.size(2))
        
        return log_p, log_det

In [11]:
model = WaveGlow().to(device)
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1937.)
  """


In [ ]:
iteration = 0

for epoch in range(100):
  print("Epoch: {}".format(epoch))
  for i, (x, c) in enumerate(train_loader):
    start = time.perf_counter()
    model.zero_grad()
    x, c = x.to(device), c.to(device)
    log_p, logdet = model(x, c)
    log_p, logdet = torch.mean(log_p), torch.mean(logdet)

    loss = -(log_p + logdet)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 1.)
    
    optimizer.step()

    duration = time.perf_counter() - start
    print("Iteratoin: {}, [NLL, Log p(z), Log Det]: [{:.4f}, {:.4f}, {:.4f}], {:.2f}s/it".format(iteration, loss.item(), log_p.item(), logdet.item(), duration))
    
    iteration += 1

Epoch: 0
Iteratoin: 0, [NLL, Log p(z), Log Det]: [0.9203, -0.9203, 0.0000], 3.39s/it
Iteratoin: 1, [NLL, Log p(z), Log Det]: [0.7997, -0.9214, 0.1217], 3.34s/it
Iteratoin: 2, [NLL, Log p(z), Log Det]: [0.2797, -1.0120, 0.7323], 3.34s/it
Iteratoin: 3, [NLL, Log p(z), Log Det]: [33.4873, -36.2029, 2.7155], 3.34s/it
Iteratoin: 4, [NLL, Log p(z), Log Det]: [267.4579, -270.4386, 2.9807], 3.35s/it
Iteratoin: 5, [NLL, Log p(z), Log Det]: [35.9919, -38.4573, 2.4654], 3.35s/it
Iteratoin: 6, [NLL, Log p(z), Log Det]: [5.2407, -6.9505, 1.7098], 3.33s/it
Iteratoin: 7, [NLL, Log p(z), Log Det]: [0.8842, -1.6817, 0.7974], 3.33s/it
Iteratoin: 8, [NLL, Log p(z), Log Det]: [0.9622, -0.9956, 0.0334], 3.32s/it
Iteratoin: 9, [NLL, Log p(z), Log Det]: [0.2120, -1.2408, 1.0288], 3.33s/it
Iteratoin: 10, [NLL, Log p(z), Log Det]: [0.5717, -2.5050, 1.9334], 3.32s/it
Iteratoin: 11, [NLL, Log p(z), Log Det]: [0.4569, -2.8644, 2.4075], 3.32s/it
Iteratoin: 12, [NLL, Log p(z), Log Det]: [0.3157, -2.7200, 2.4043], 3